In [1]:
import os
ON_KAGGLE = os.path.exists('/kaggle/input')
ENABLE_SUBMISSION = False
GENERATE_WHL = False
os.environ["ON_KAGGLE"] = str(ON_KAGGLE)

In [2]:
# Install required packages if on Kaggle
if ON_KAGGLE:
    !pip install -U /kaggle/input/accelerate-wheelwhl/accelerate-0.29.1-py3-none-any.whl -qq
    !pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/hydracore120py3/hydra_core-1.2.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/omegaconf222py3/omegaconf-2.2.2-py3-none-any.whl -qq
    !pip install -U /kaggle/input/parglare-0-18-0/parglare-0.18.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/rank-bm25-whl/rank_bm25-0.2.2-py3-none-any.whl -qq
    !pip install -U /kaggle/input/peft-0-11-1/peft-0.11.1-py3-none-any.whl -qq
    !pip install -U /kaggle/input/trl-0-9-4/trl-0.9.4-py3-none-any.whl -qq
    !pip install -U /kaggle/input/amino-gaz-whl/aimo_gaz-0.0.1-py3-none-any.whl -qq --no-deps --force-reinstall
else:
    !pip install -r requirements.txt
    !pip install -e . --force-reinstall
if GENERATE_WHL:
    !pip install --upgrade build
    !python -m build

In [6]:
import time
from aimo_gaz.solver.solver_config import parse_solver_config
from aimo_gaz.tools.log_utils import setup_logger
from hydra import compose, initialize

config_path = '/kaggle/input/aimo-gaz-configs/configs' if ON_KAGGLE else 'src/aimo_gaz/configs'
config_name = 'coordination_solver_config.yaml'

# Setup the logger
if "AIMO_GAZ_ROOT" not in os.environ:
    os.environ["AIMO_GAZ_ROOT"] = '/kaggle/working' if ON_KAGGLE else 'src/aimo_gaz'
log_dir_name = "logs" if ON_KAGGLE else '.logs'
root_dir = os.environ["AIMO_GAZ_ROOT"]
# Check if the configs exists in the root directory
if not os.path.exists(os.path.join(root_dir, 'configs')):
    os.makedirs(os.path.join(root_dir, 'configs'))
# Copy the configs to the root directory
os.system(f'cp -r {config_path}/* {os.path.join(root_dir, "configs")}')
# Create the logs directory
if not os.path.exists(os.path.join(root_dir, log_dir_name)):
    os.makedirs(os.path.join(root_dir, log_dir_name), exist_ok=True)
time_str = time.strftime("%Y-%m-%d_%H-%M-%S")
os.makedirs(os.path.join(root_dir, log_dir_name, time_str), exist_ok=True)
logger = setup_logger("aimo_gaz", os.path.join(root_dir, log_dir_name, time_str, 'aimo_gaz.log'))
logger.info(f"Using config file: {config_name}")
with initialize(version_base="1.2", config_path="configs", job_name="aimo_gaz"):
    cfg = compose(config_name=config_name) # Load the config file
    solver_config = parse_solver_config(cfg)
solver = solver_config.get_solver(logger=logger)
assert solver is not None, "Solver not initialized"
    

In [14]:
# Set up the evaluation API
if ENABLE_SUBMISSION and ON_KAGGLE:
    import aimo

    env = aimo.make_env()
    iter_test = env.iter_test()

    with solver:
        # Iterate through the test set and use the model make predictions
        for test, sample_submission in iter_test:
            sample_submission['answer'] = solver.solve(test['problem'])
            env.predict(sample_submission)
            print(test)
            print(sample_submission, '\n')